In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
import multiprocessing as mp
import pandas as pd
import pickle as pkl
import urllib

from astropy.io import fits
from astropy.table import Table
from astropy.nddata import StdDevUncertainty
from astropy.convolution import convolve
from astroquery.mast import Observations
from glob import glob
from lmfit import Model
from lmfit.printfuncs import report_fit
from lmfit import conf_interval, report_ci, minimize
from pathlib import Path
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
from specutils.manipulation import LinearInterpolatedResampler
from specutils import Spectrum1D

# Load in data (downloaded in scratch.ipynb using MAST archive API)

In [8]:
# Load in RU Lupi data
spectrum_files = glob(os.path.join('HST','anonymous37222', '*x1dsum.fits'))

In [10]:
spec = Table.read(spectrum_files[1])
spec['WAVELENGTH'][spec['SEGMENT']=='FUVA'].min()

np.float64(1287.6826131868756)

In [11]:
INST_head = [fits.getheader(file, ext=0) for file in spectrum_files]
SCI_head = [fits.getheader(file, ext=1) for file in spectrum_files]
spectrum_x1d = [Table.read(file, hdu=1) for file in spectrum_files]

In [12]:
#from each spectrum, function to get specific lines OR specific transitions from France emission_lines.csv
lines = pd.read_csv('HST/emission_lines.csv', skiprows=2)

In [13]:
file_idx = 5
flux_units = (u.erg/(u.AA*u.s*(u.cm**-2)))

x1d_wavelength = np.ravel(spectrum_x1d[file_idx]['WAVELENGTH'])
x1d_flux = np.ravel(spectrum_x1d[file_idx]['FLUX'])
x1d_fluxerr = np.ravel(spectrum_x1d[file_idx]['ERROR'])

In [14]:
peaks , _ = find_peaks(np.array(x1d_flux), height=max(x1d_fluxerr.value)*2, distance=100)
#Find the emission lines associated with H2 fluorescence
emission_table_idx = []
lab_line_row = []
for i, l in enumerate(lines['lab_lambda']):
    diff = abs(x1d_wavelength[peaks]/u.AA-l)
    if min(diff) < 5:
        emission_table_idx.append(np.argmin(diff))
        lab_line_row.append(lines.iloc[i])

In [ ]:
datadir = 'HST/'
def fetch_COSfiles(grating, lpPos, cenwave, disptab):

    """
    Given all the inputs: (detector, grating, LP-POS, cenwave, dispersion table,) this will download both
    the LSF file and Disptab file you should use in the convolution and return their paths.
    Returns:
    LSF_file_name (str): filename of the new downloaded LSF file
    disptab_path (str): path to the new downloaded disptab file
    """
    COS_site_rootname = (
        "https://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/cos/"
        "performance/spectral-resolution/_documents/"
    )  # Link to where all the files live - split into 2 lines
    LSF_file_name = f"aa_LSFTable_{grating}_{cenwave}_LP{lpPos}_cn.dat"

    LSF_file_webpath = COS_site_rootname + LSF_file_name  # Where to find file online
    lsffileSave = datadir+LSF_file_name
    lsfpathObj = Path(lsffileSave)
    if not lsfpathObj.is_file():
        try:
            urllib.request.urlretrieve(
                LSF_file_webpath, lsffileSave
            )  # Where to save file to locally
            print(f"Downloaded LSF file to {lsffileSave}")
        except urllib.error.HTTPError:
            print(f'404 error for {lsffileSave}')
    else:
        print(f'File already exists, skipping {lsffileSave}')

    # And we'll need to get the DISPTAB file as well
    disptab_path = datadir + disptab
    disptabpathObj = Path(disptab_path)
    if not disptabpathObj.is_file():
        try:
            urllib.request.urlretrieve(
                f"https://hst-crds.stsci.edu/unchecked_get/references/hst/{disptab}",
                disptab_path,
            )
            print(f"Downloaded DISPTAB file to {disptab_path}")
        except urllib.error.HTTPError:
            print(f'404 error for {disptab_path}')
    else:
        print(f'File already exists, skipping {disptab_path}')
    return LSF_file_name, disptab_path